In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

We've already provided the code needed to load the data, perform iterative parsing and write the
output to csv files. Your task is to complete the shape_element function that will transform each
element into the correct format. To make this process easier we've already defined a schema (see
the schema.py file in the last code tab) for the .csv files and the eventual tables. Using the 
cerberus library we can validate the output against this schema to ensure it is correct.

## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
          'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
          'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""

import csv
import codecs
import re
import xml.etree.cElementTree as ET
from collections import OrderedDict
import cerberus

import schema

OSM_PATH = "example.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'user', 'uid', 'version','lat','lon', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for a in element.attrib:
            if a in NODE_FIELDS:
                if a not in node_attribs:

                    node_attribs[a]=element.attrib[a]
        
#   <tag k="addr:street:name" v="Lincoln"/>
#   should be turned into
#   {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}       

        for tag in element.iter('tag'):
            if not PROBLEMCHARS.search(tag.attrib['k']):
                
                    
                each_tag={}
                each_tag['id']=node_attribs['id']
                each_tag['value']=tag.attrib['v']
                if LOWER_COLON.search(tag.attrib['k']):
                    
                    sub_attrib=tag.attrib['k'].split(':',1)
                        
                    each_tag['key']=sub_attrib[1]

                    each_tag['type']=sub_attrib[0]
                else:
                    each_tag['key']=tag.attrib['k']
                    each_tag['type']='regular'
                    
        
                key_order= ('id', 'key', 'value', 'type')
                OrderedDict((k, each_tag[k]) for k in key_order)
                tags.append(each_tag)
           
        
        
        
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for a in element.attrib:
            if a in WAY_FIELDS:
                if a not in way_attribs:

                    way_attribs[a]=element.attrib[a]
        count=0
        for nd in element.iter('nd'):
            
            each_way_node={}
            
            
            each_way_node['position']=count
            count+=1
            each_way_node['node_id']=nd.attrib['ref']
            each_way_node['id']=way_attribs['id']
            way_nodes.append(each_way_node)

        
        for tag in element.iter('tag'):
            if not PROBLEMCHARS.search(tag.attrib['k']):
                
                    
                each_tag={}
                each_tag['id']=way_attribs['id']
                each_tag['value']=tag.attrib['v']
                if LOWER_COLON.search(tag.attrib['k']):
                    
                    sub_attrib=tag.attrib['k'].split(':',1)
                        
                    each_tag['key']=sub_attrib[1]

                    each_tag['type']=sub_attrib[0]
                else:
                    each_tag['key']=tag.attrib['k']
                    each_tag['type']='regular'
                    
        
                tags.append(each_tag)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


In [6]:
import numpy as np
import pandas as pd
import json
import pandas.io.data as web
from datetime import date, datetime, timedelta
from collections import defaultdict
start = datetime(2010, 1, 1)
end = date.today()

c = web.DataReader('aapl', 'yahoo', start, end)
print type(c)

<class 'pandas.core.frame.DataFrame'>


In [34]:
out_df=pd.DataFrame(index=[0])
out_df['eight_new_price']=True


In [35]:
out_df

,eight_new_price
0,True


In [29]:
class Complex:
    def __init__(self,data):
        self.data=data
        out_df=pd.DataFrame(index=[0])

In [30]:
def output(self):
    
    out_df[eight_new_price]=True
    return out_df
#     dragonfly_doji(data_pt)
#     gravestone_doji(data_pt)
#     long_legged_doji(data_pt)
#     body_candle(data_pt)
#     black_candle(data_pt)
#     tall_black_candle(data_pt)
#     small_black_candle(data_pt)
#     white_candle(data_pt)
#     tall_white_candle(data_pt)
#     small_white_candle(data_pt)
#     white_marubozu_candle(data_pt)
#     black_marubozu_candle(data_pt)
#     closing_black_marubozu_candle(data_pt)
#     closing_white_marubozu_candle(data_pt)
#     black_spinning_top_candle(data_pt)
#     black_spinning_top_candle(data_pt)
#    up_price_trend(data_pt, data_pt1, data_pt2)
#    down_price_trend(data_pt, data_pt1, data_pt2)
#    similar_price(data_pt1,data_pt2, percent = 0.001)

eight_new_price(data)
ten_new_price(data)
twelve_new_price(data)
thirteen_new_price(data)
bearish_abandoned_baby(data)
bullish_abandoned_baby(data)
above_stomach(data)
advance_block(data)
below_stomach(data)
bearish_belt_hold(data)
bearish_breakaway(data)
bearish_doji_star(data)
bearish_engulfing(data)
bearish_harami(data)
bearish_harami_cross(data)
bearish_kicking(data)
bearish_meeting_lines(data)
bearish_separating_lines(data)
bearish_side_by_side_white_lines(data)
bearish_three_line_strike(data)
bearish_tri_star(data)
bullish_belt_hold(data)
bullish_breakaway(data)
bullish_doji_star(data)
bullish_engulfing(data)
bullish_harami(data)
bullish_harami_cross(data)
bullish_kicking(data)
bullish_meeting_lines(data)
bullish_separating_lines(data)
bullish_side_by_side_white_lines(data)
bullish_three_line_strike(data)
bullish_tri_star(data)
collapsing_doji_star(data)
conceling_baby_swallow(data)
dark_cloud_cover(data)
deliberation(data)
gapping_down_doji(data)
gapping_up_doji(data)
northern_doji(data)
southern_doji(data)
bearish_doji_star(data)
bullish_doji_star(data)
evening_doji(data)
downside_gap_three_methods(data)
downside_tasuki_gap(data)
falling_three_methods(data)
falling_window(data)
hammer(data)
inverted_hammer(data)
hanging_man(data)
high_wave(data)
homing_pigeon(data)
identical_three_crows(data)
in_neck(data)
ladder_bottom(data)
last_engulfing_bottom(data)
last_engulfing_top(data)
matching_low(data)
mat_hold(data)
morning_doji_star(data)
morning_star(data)
on_neck(data)
piercing_pattern(data)
rickshaw_man(data)
rising_three_methods(data)
rising_window(data)
shooting_star_1(data)
shooting_star_2(data)
stick_sandwich(data)
takuri_line(data)
three_black_crows(data)
three_inside_down(data)
three_inside_up(data)
three_outside_down(data)
three_outside_up(data)
three_stars_in_south(data)
three_white_soldiers(data)
thrusting(data)
tweezers_bottom(data)
tweezers_top(data)
two_black_gapping(data)
two_crows(data)
unique_three_river_bottom(data)
upside_gap_three_methods(data)
upside_gap_two_crows(data)
upside_tasuki_gap(data)

In [26]:
import numpy as np 
import pandas as pd 
import json
import pandas.io.data as web
from datetime import date, datetime, timedelta
from collections import defaultdict

class td_sequence(object):
    def __init__(self, data):
        self.data = data

    def sequence(self):
        setup = self.data.iloc[-1]['Close'] - self.data.iloc[-1-4]['Close']
        buy_setup = True
        buy_counter = 1
        sell_counter = -1
        if setup < 0:
            '''buy setup'''
            buy_setup = True
        elif setup > 0:
            '''sell setup'''
            buy_setup = False
        for i in xrange(1,(len(self.data))):
            if buy_setup:
                buy = self.data.iloc[-1-i]['Close'] - self.data.iloc[-5-i]['Close']
                if buy < 0:
                    buy_counter += 1
                    if buy_counter > 9:
                        '''failed to reverse, reset buy counter back to 1'''
                        buy_counter = 1 
                    if buy_counter == 9 and ((self.data.iloc[-2-i]['Close'] - self.data.iloc[-6-i]['Close'])>0):
                        if ((self.data.iloc[-1]['Low'] <= self.data.iloc[-3]['Low']) and (self.data.iloc[-1]['Low'] <= self.data.iloc[-4]['Low'])) or \
                            ((self.data.iloc[-2]['Low'] <= self.data.iloc[-3]['Low']) and (self.data.iloc[-2]['Low'] <= self.data.iloc[-4]['Low'])):
                            buy_counter = 10
                            return buy_counter
                        else:
                            return buy_counter
                else:
                    if (buy_counter == 8) and ((self.data.iloc[-2]['Low'] <= self.data.iloc[-3]['Low']) and (self.data.iloc[-2]['Low'] <= self.data.iloc[-4]['Low'])):
                        buy_counter = 8.5
                        return 8.5
                    else:
                        return buy_counter
            else:
                sell = self.data.iloc[-1-i]['Close'] - self.data.iloc[-5-i]['Close']
                if sell > 0:
                    sell_counter -= 1
                    if sell_counter < -9:
                        '''failed to reverse, reset buy counter back to -1'''
                        sell_counter = -1 
                    if sell_counter == -9 and ((self.data.iloc[-2-i]['Close'] - self.data.iloc[-6-i]['Close'])<0):
                        if ((self.data.iloc[-1]['High'] > self.data.iloc[-3]['High']) and (self.data.iloc[-1]['High'] > self.data.iloc[-4]['High'])) or \
                            ((self.data.iloc[-2]['High'] > self.data.iloc[-3]['High']) and (self.data.iloc[-2]['High'] > self.data.iloc[-4]['High'])):
                            sell_counter = -10
                            return sell_counter
                        else:
                            return sell_counter
                else:
                    if sell_counter == -8 and ((self.data.iloc[-2]['High'] > self.data.iloc[-3]['High']) and (self.data.iloc[-2]['High'] > self.data.iloc[-4]['High'])):
                        sell_counter = -8.5
                        return -8.5
                    else:
                        return sell_counter
        

In [27]:
import pandas as pd 
import numpy as np 
import datetime as dt
import pandas.io.data as web
from decimal import Decimal
from td_sequence import TDSequence
'''
Sample data from 2016-07-25, name may changes as date changes
Using AAPL as test
'''
symbol = 'AAPL'
start = dt.datetime(2012, 1, 1)
end = dt.date.today()
'''DF details from before'''
er_date = '2016-08-16'
c = web.DataReader(symbol, 'yahoo', start, end)
df_stock_detail = pd.read_pickle('data/2016-07-25_all_stocks_info')
df_stock_cash_flow_quarter = pd.read_pickle('data/2016-08-19_cash_flow_quarter.pkl')
df_stock_balance_sheet_quarter = pd.read_pickle('data/2016-08-19_balance_sheet_quarter.pkl')
df_stock_balance_sheet_annual = pd.read_pickle('data/2016-08-19_balance_sheet_annual.pkl')
df_stock_income_statement_quarter = pd.read_pickle('data/2016-08-19_income_statement_quarter.pkl')
df_stock_income_statement_annual = pd.read_pickle('data/2016-08-19_income_statement_quarter.pkl')

df_er = pd.read_pickle('data/rev_full_history_er_date.pkl')

new_col_names = []
for i in df_stock_detail.columns.values:
    new_col_names.append(i.strip())
df_stock_detail.columns = new_col_names

'''sample data detail: AAPL'''
c = web.DataReader(symbol, 'yahoo', start, end)
earning_report = df_er[df_er['symbol'] == symbol][:2]
earning_report['er_date'] = pd.to_datetime(earning_report['er_date'])
earning_report['adj_er_date'] = earning_report['er_date'] if ('After' in earning_report['time'] or 'pm' in earning_report['time'])else earning_report['er_date']-dt.timedelta(days=1)
earning_report.reset_index(drop = True, inplace = True)

for i in xrange(len(earning_report)):
    price = Decimal(c.loc[earning_report['adj_er_date'][i+1]]['Close'])
    symbol_cash_flow_q = df_stock_cash_flow_quarter[df_stock_cash_flow_quarter['symbol'] == symbol]
    symbol_balance_sheet_q = df_stock_balance_sheet_quarter[df_stock_balance_sheet_quarter['symbol'] == symbol]
    symbol_income_statement_q = df_stock_income_statement_quarter[df_stock_income_statement_quarter['symbol'] == symbol]
    symbol_cash_flow_prevQ1 = symbol_cash_flow_q.iloc[i+1]
    symbol_cash_flow_prevQ2 = symbol_cash_flow_q.iloc[i+2]
    symbol_balance_sheet_prevQ1 = symbol_balance_sheet_q.iloc[i+1]
    symbol_balance_sheet_prevQ2 = symbol_balance_sheet_q.iloc[i+2]
    symbol_income_statement_prevQ1 = symbol_income_statement_q.iloc[i+1]

    '''fundamental details'''
    preferred_stock = Decimal(0 if symbol_balance_sheet_prevQ1['Redeemable Preferred Stock, Total'] == None else symbol_balance_sheet_prevQ1['Redeemable Preferred Stock, Total']) \
                        + Decimal(0 if symbol_balance_sheet_prevQ1['Preferred Stock - Non Redeemable, Net'] == None else symbol_balance_sheet_prevQ1['Preferred Stock - Non Redeemable, Net']) 
    book_value = (symbol_balance_sheet_prevQ1['Total Equity'] - preferred_stock ) / symbol_balance_sheet_prevQ1['Total Common Shares Outstanding']
    free_cash_flow = (symbol_cash_flow_prevQ1['Cash from Operating Activities'] + symbol_cash_flow_prevQ1['Capital Expenditures'])
    total_revenue = symbol_income_statement_prevQ1['Total Revenue']
    net_income = symbol_income_statement_prevQ1['Net Income']
    debt = symbol_balance_sheet_prevQ1['Total Long Term Debt']
    equity = symbol_balance_sheet_prevQ1['Total Equity']

    '''value investing'''
    price_to_book_value_q = price/book_value
    price_to_free_cash_flow_q = price/free_cash_flow
    price_to_earnings_q = price / symbol_income_statement_prevQ1['Diluted Normalized EPS']
    price_to_sales_q = total_revenue / symbol_balance_sheet_prevQ1['Total Common Shares Outstanding']
    dividends = symbol_income_statement_prevQ1['Dividends per Share - Common Stock Primary Issue']
    long_term_debt_quarter = symbol_balance_sheet_prevQ1['Long Term Debt']
    # long_term_debt_annual = symbol_balance_sheet_annual['Long Term Debt']
    if max(symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'],symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']) / min(symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'], symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']) >= 2:
        capital_spending_diff = symbol_cash_flow_prevQ1['Capital Expenditures']/symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'] - symbol_cash_flow_prevQ2['Capital Expenditures']/symbol_balance_sheet_prevQ1['Total Common Shares Outstanding']
    else:
        capital_spending_diff = symbol_cash_flow_prevQ1['Capital Expenditures']/symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'] - symbol_cash_flow_prevQ2['Capital Expenditures']/symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']
    market_cap = symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'] * price
    return_on_total_capital =  (net_income - dividends * symbol_balance_sheet_prevQ1['Total Common Shares Outstanding']) / (debt + equity)
    return_on_shareholders_equity = net_income/equity 
    extra_shares_outstanding = symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'] - symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']

    '''td_sequential'''
    end_date = earning_report['adj_er_date'][i]
    data = web.DataReader(symbol, 'yahoo', start, end_date)
    td = TDSequence(data)
    td_sequence = td.sequence()
    '''candle_stick'''
# Below are for futrue test data...not useful for training with lack of data
# symbol_detail = df_stock_detail[df_stock_detail['Symbol'] == symbol]
# symbol_quarter_idx = symbol+'_prev_1Q'
# temp = symbol+'_' + str(dt.date.today().year)
# temp2 = symbol+'_' + str(dt.date.today().year-1)
# if temp in df_stock_balance_sheet_annual.index:
#   symbol_annual_idx = temp
# elif temp2 in df_stock_balance_sheet_annual.index:
#     symbol_annual_idx = temp2
# else:
#     symbol_annual_idx = symbol+'_' + str(dt.date.today().year-2)

# symbol_cash_flow_prevQ1 = df_stock_cash_flow_quarter.ix[symbol_quarter_idx]
# symbol_balance_sheet_annual = df_stock_balance_sheet_annual.ix[symbol_annual_idx]
# symbol_balance_sheet_prevQ1 = df_stock_balance_sheet_quarter.ix[symbol_quarter_idx]
# symbol_balance_sheet_prevQ2 = df_stock_balance_sheet_quarter.ix[symbol+'_prev_2Q']
# symbol_cash_flow_prevQ1 = df_stock_cash_flow_quarter.ix[symbol_quarter_idx]
# symbol_cash_flow_prevQ2 = df_stock_cash_flow_quarter.ix[symbol+'_prev_2Q']

# '''value investing'''
# price_to_book_value = symbol_detail['Price/Book']
# price_to_free_cash_flow_prevQ1 = c['Adj Close'][er_date]/(symbol_cash_flow_prevQ1['Cash from Operating Activities'] + symbol_cash_flow_prevQ1['Capital Expenditures'])
# price_to_earnings = symbol_detail['P/E Ratio']
# price_to_sales = symbol_detail['Price/Sales']
# dividends = symbol_detail['Dividend/Share']
# long_term_debt_quarter = symbol_balance_sheet_prevQ1['Long Term Debt']
# long_term_debt_annual = symbol_balance_sheet_annual['Long Term Debt']

# if max(symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'],symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']) / min(symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'], symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']) >= 2:
#     capital_spending_diff = symbol_cash_flow_prevQ1['Capital Expenditures']/symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'] - symbol_cash_flow_prevQ2['Capital Expenditures']/symbol_balance_sheet_prevQ1['Total Common Shares Outstanding']
# else:
#     capital_spending_diff = symbol_cash_flow_prevQ1['Capital Expenditures']/symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'] - symbol_cash_flow_prevQ2['Capital Expenditures']/symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']

# market_cap = symbol_detail['Market Capitalization']
# shares_outstanding = symbol_balance_sheet_prevQ1['Total Common Shares Outstanding'] - symbol_balance_sheet_prevQ2['Total Common Shares Outstanding']

ImportError: No module named td_sequence